I use three models：
==
LogisticRegression

LinearSVC

XGBClassifier

Use their predictions as a new feature:(logis_pred， svc_pred， xgb_pred）
Then tarin a XGBClassifier

The validation score 
train score 0.8503685147840804
val score 0.8254763097498596

But the effect is very poor on the test set

Can someone tell me why？

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')

In [ ]:
x_data = train.drop(['id', 'target'], axis=1)
y_data = train.target 
x_test = test.drop('id', axis=1)

In [ ]:
null_count = pd.isnull(x_data).sum().sum()
print(f'null data, {null_count}')

In [ ]:
x_data.describe()

In [ ]:
x_data.info()

In [ ]:
x_data.head()

In [ ]:

plt.hist(x_data.std())

In [ ]:
x_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_data)
x_data = scaler.transform(x_data)
x_test = scaler.transform(x_test)

In [ ]:
from sklearn.metrics import roc_auc_score

## model trainning

### LogisticRegression

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.5)

In [ ]:
from sklearn.linear_model import LogisticRegression

logis_model = LogisticRegression()
logis_model.fit(x_train, y_train)

logis_train_pred = logis_model.predict_proba(x_train)[:, 1]
score = roc_auc_score(y_train, logis_train_pred)
print(f'train score {score}')

logis_val_pred = logis_model.predict_proba(x_val)[:, 1]
score = roc_auc_score(y_val, logis_val_pred)
print(f'val score {score}')

# logis_pred = logis_model.predict_proba(x_data)[:, 1]
# print(f'logis pred {logis_pred.shape}')

### LinearSVC

In [ ]:
from sklearn.svm import LinearSVC

# x_train, x_val, y_train, y_val = train_test_split(x_data, y_data)
svc_model = LinearSVC()
svc_model.fit(x_train, y_train)

svc_train_pred = svc_model.decision_function(x_train)
score = roc_auc_score(y_train, svc_train_pred)
print(f'train score {score}')

svc_val_pred = svc_model.decision_function(x_val)
score = roc_auc_score(y_val, svc_val_pred)
print(f'val score {score}')

# svc_pred = svc_model.decision_function(x_data)
# print(f'logis pred {svc_pred.shape}')

### XGBClassifier

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_params = {
                'n_estimators': 4492,
                'learning_rate': 0.01,
                'subsample': 1.0,
                'colsample_bytree': 0.2,
                'max_depth': 15,
                'gamma': 1.0328829988080024,
                'reg_alpha': 100,
                'reg_lambda': 93 }

xgb_params['tree_method'] = 'gpu_hist'
xgb_params['predictor'] = 'gpu_predictor'
xgb_params['use_label_encoder'] = False

In [ ]:
# x_train, x_val, y_train, y_val = train_test_split(x_data, y_data)
xgb_model = XGBClassifier(**xgb_params)
xgb_model.fit(x_train, y_train)

xgb_train_pred = xgb_model.predict_proba(x_train)[:, 1]
score = roc_auc_score(y_train, xgb_train_pred)
print(f'train score {score}')

xgb_val_pred = xgb_model.predict_proba(x_val)[:, 1]
score = roc_auc_score(y_val, xgb_val_pred)
print(f'val score {score}')

# xgb_pred = xgb_model.predict_proba(x_data)[:, 1]
# print(f'logis pred {svc_pred.shape}')

### logis_pred svc_pred xgb_pred as a new feature

In [ ]:
new_x_train_data = np.hstack([logis_train_pred[:, np.newaxis], svc_train_pred[:, np.newaxis], xgb_train_pred[:, np.newaxis]])
print(f'new_x_data shape {new_x_train_data.shape}')

new_x_train_data = np.hstack([x_train, new_x_train_data])

new_x_val_data = np.hstack([logis_val_pred[:, np.newaxis], svc_val_pred[:, np.newaxis], xgb_val_pred[:, np.newaxis]])
print(f'new_x_data shape {new_x_val_data.shape}')

new_x_val_data = np.hstack([x_val, new_x_val_data])

### train xgboost

In [ ]:
# x_train, x_val, y_train, y_val = train_test_split(new_x_data, y_data)
model = XGBClassifier(**xgb_params)
model.fit(new_x_val_data, y_val)

train_pred = model.predict_proba(new_x_train_data)[:, 1]
score = roc_auc_score(y_train, train_pred)
print(f'train score {score}')

val_pred = model.predict_proba(new_x_val_data)[:, 1]
score = roc_auc_score(y_val, val_pred)
print(f'val score {score}')

In [ ]:
new_x_data = np.vstack([new_x_train_data, new_x_val_data])
print(new_x_data.shape)

new_y_data = np.concatenate([y_train, y_val])
print(new_y_data.shape)

In [ ]:
# from sklearn.linear_model import Ridge

In [ ]:
# x_train, x_val, y_train, y_val = train_test_split(new_x_data, y_data)
# model = LogisticRegression()
# model.fit(x_train, y_train)

# train_pred = model.predict(x_train)
# score = roc_auc_score(y_train, train_pred)
# print(f'train score {score}')

# val_pred = model.predict(x_val)
# score = roc_auc_score(y_val, val_pred)
# print(f'val score {score}')

### predict

In [ ]:
logis_test_pred = logis_model.predict_proba(x_test)[:, 1]
svc_test_pred = svc_model.decision_function(x_test)
xgb_test_pred = xgb_model.predict_proba(x_test)[:, 1]

new_x_test = np.hstack([logis_test_pred[:, np.newaxis], svc_test_pred[:, np.newaxis], xgb_test_pred[:, np.newaxis]])
new_x_test = np.hstack([x_test, new_x_test])

y_test = model.predict_proba(new_x_test)[:, 1]

In [ ]:
# from sklearn.model_selection import KFold
# n_split = 10
# kfold = KFold(n_split)
# y_test = np.zeros(shape=(new_x_test.shape[0],))
# for i, (train_index, val_index) in enumerate(kfold.split(new_x_train_data)):
#     print(f'fold {i}')
    
#     x_train = new_x_data[train_index]
#     y_train = new_y_data[train_index]
    
#     x_val = new_x_data[val_index]
#     y_val = new_y_data[val_index]
    
#     model = XGBClassifier(**xgb_params)
#     model.fit(x_train, y_train)
    
#     train_pred = model.predict_proba(x_train)[:, 1]
#     score = roc_auc_score(y_train, train_pred)
#     print(f'train score {score}')

#     val_pred = model.predict_proba(x_val)[:, 1]
#     score = roc_auc_score(y_val, val_pred)
#     print(f'val score {score}')
    
#     y_test += model.predict_proba(new_x_test)[:, 1] / n_split

In [ ]:
sub_mission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')
sub_mission.target = y_test
sub_mission.to_csv('submission.csv', index=False)

plt.figure()
sns.kdeplot(y_test)